In [1]:
import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import matplotlib
matplotlib.use('SVG')

import helper.callbacks
import helper.model_builder
import helper.visualize
import helper.data_provider
import helper.metrics

import skimage.io
import sklearn.metrics

import scipy.stats
import pandas as pd

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

import sys

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
# constants
const_lr = 1e-4

tag = '01'

# Output dirs
out_dir = '/data1/image-segmentation/BBBC022/unet/experiments/' + tag + '/out'
tb_log_dir = "/data1/image-segmentation/BBBC022/unet/tensorboard/" + tag + "/"
chkpt_dir = "/data1/image-segmentation/BBBC022/unet/experiments/" + tag + "/checkpoints/"

os.makedirs(out_dir, exist_ok=True)
os.makedirs(tb_log_dir, exist_ok=True)
os.makedirs(chkpt_dir, exist_ok=True)

# Files
chkpt_file = chkpt_dir + "{epoch:04d}.hdf5"
csv_log_file = "/data1/image-segmentation/BBBC022/unet/experiments/" + tag + "/log.csv"

# Input dirs
train_dir_x = '/data1/image-segmentation/BBBC022/unet/split/training/x/'
train_dir_y = '/data1/image-segmentation/BBBC022/unet/split/training/y/'
val_dir = "/data1/image-segmentation/BBBC022/unet/split/validation/"

# Learning Settings
rescale_labels = False

epochs = 2

batch_size = 4
steps_per_epoch = 2 * 4 / batch_size

# make sure these number for to the validation set
val_batch_size = 10
val_steps = int(50 * 4 / val_batch_size)

# generator only params
dim1 = 256
dim2 = 256

bit_depth = 8

In [3]:
# build session running on GPU 1
configuration = tf.ConfigProto()
#configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = ""
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = helper.data_provider.random_sample_generator(
    train_dir_x,
    train_dir_y,
    batch_size,
    bit_depth,
    dim1,
    dim2,
    rescale_labels
)

val_gen = helper.data_provider.single_data_from_images(
    val_dir + 'x/',
     val_dir + 'y/',
     val_batch_size,
     bit_depth,
     dim1,
     dim2,
     rescale_labels
)

Found 50 images belonging to 1 classes.
Found 50 images belonging to 1 classes.


In [ ]:
# build model
model = helper.model_builder.get_model_3_class(dim1, dim2)
model.summary()

loss = "categorical_crossentropy"
metrics = [keras.metrics.categorical_accuracy, helper.metrics.recall, helper.metrics.precision]

optimizer = keras.optimizers.RMSprop(lr = const_lr)

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# CALLBACKS
# save model after each epoch
callback_model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=chkpt_file,
    save_weights_only=True,
    save_best_only=False
)
callback_csv = keras.callbacks.CSVLogger(filename=csv_log_file)
# callback_splits_and_merges = helper.callbacks.SplitsAndMergesLoggerBoundary(
#     'images',
#     val_gen,
#     gen_calls = val_steps,
#     log_dir=tb_log_dir
# )

callbacks=[callback_model_checkpoint, callback_csv] #, callback_splits_and_merges]


/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:27: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:33: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jccaicedo/unet4nuclei/unet4nuc

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 1)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 256, 256, 64)  640         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 256, 256, 64)  256         conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 256, 256, 64)  36928       batch_normalization_1[0][0]      
___________________________________________________________________________________________

/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:115: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  y = keras.layers.merge([f, a], concat_axis=3, mode="concat")
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:119: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  y = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(y)
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:121: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  y = keras.layers.BatchNormalization(**option_dict_bn)(y)
/home/jccaicedo/unet4nuclei/unet4nuclei/helper/model_builder.py:125: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  y = keras.layers.Convolution2D(64, 3,

In [ ]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    callbacks=callbacks,
    verbose = 1
)

# visualize learning stats
#helper.visualize.visualize_learning_stats_boundary_hard(statistics, out_dir, metrics)

print('Done! :)')

Epoch 1/2
Found 100 images.
Found 100 annotations.
2/2 [==============================] - 131s - loss: 262.9672 - categorical_accuracy: 0.5700 - recall: 0.9998 - precision: 1.8817 - val_loss: 255.0504 - val_categorical_accuracy: 0.7558 - val_recall: 1.0000 - val_precision: 6553600000000.0000
Epoch 2/2
1/2 [==============>...............] - ETA: 7s - loss: 141.3766 - categorical_accuracy: 0.8868 - recall: 0.9998 - precision: 1.3797